In [ ]:
from fastdebug.core import *

### Make sure all necessary variables are available 

In [ ]:
import inspect
from inspect import *
from inspect import _signature_from_callable
from inspect import _signature_is_functionlike, _signature_is_builtin, _signature_get_user_defined_method, _signature_from_function, _signature_bound_method

In [ ]:
class Foo:
    def __init__(self, a, b, c): pass

_signature_from_callable(Foo, sigcls=inspect.Signature, follow_wrapper_chains=True) # debug the function above through this example

<Signature (a, b, c)>

### give dbprint the current main globals() is very crucial

In [ ]:
g = globals() # this is a crucial step
# 1. it brings everything _signature_from_callable needed from inspect.py 
# 2. it allows the new srcode function override the original official srcode in outer scope, 
# so that you can call the new function with the same name _signature_from_callable

### Try one srcline with one investigation

In [ ]:
# see the second line of the params of the example below
dbprintinsert??

Signature: dbprintinsert(*codes, env={})
Docstring: <no docstring>
Source:   
def dbprintinsert(*codes, env={}): 
    for c in codes:
    # print(f"{c} => {c} : {eval(c, globals().update(env))}") 
        output = f"{c} => {c} : {eval(c, globals().update(env))}"
        print('{:>157}'.format(output))
File:      ~/Documents/fastdebug/fastdebug/core.py
Type:      function


In [ ]:
sfc = dbprint(_signature_from_callable, "if isinstance(obj, type):", "this is comment", \
              "obj", "isinstance(obj, type)", "env=g", \
              expand=1, env=g)

_signature_from_callable(Foo, sigcls=inspect.Signature, follow_wrapper_chains=True) # debug the function above through this example

    sig = None                                                                                                                                               
    if isinstance(obj, type):================================================================================================================================
                                                                                                                                              this is comment
        # obj is a class or a metaclass                                                                                                                      
                                                                                                                          obj => obj : <class '__main__.Foo'>
                                                                                                        isinstance(obj, type) => isinstance(obj, type) : True


<Signature (a, b, c)>

In [ ]:
_signature_from_callable?

Signature:
_signature_from_callable(
    obj,
    *,
    follow_wrapper_chains=True,
    skip_bound_arg=True,
    sigcls,
)
Docstring:
Private helper function to get signature for arbitrary
callable objects.
File:      Dynamically generated function. No source code available.
Type:      function


In [ ]:
defaults.orisrc?

Signature:
defaults.orisrc(
    obj,
    *,
    follow_wrapper_chains=True,
    skip_bound_arg=True,
    sigcls,
)
Docstring:
Private helper function to get signature for arbitrary
callable objects.
File:      ~/mambaforge/lib/python3.9/inspect.py
Type:      function


In [ ]:
sfc?

Signature: sfc(obj, *, follow_wrapper_chains=True, skip_bound_arg=True, sigcls)
Docstring:
Private helper function to get signature for arbitrary
callable objects.
File:      Dynamically generated function. No source code available.
Type:      function


In [ ]:
sfc(Foo, sigcls=inspect.Signature, follow_wrapper_chains=True) # debug the function above through this example

                                                                                                                          obj => obj : <class '__main__.Foo'>
                                                                                                        isinstance(obj, type) => isinstance(obj, type) : True


<Signature (a, b, c)>

### Try another srcline with more investigations

In [ ]:
sfc = dbprint(_signature_from_callable, "call = _signature_get_user_defined_method(type(obj), '__call__')", "this is comment", \
              "obj", "_signature_get_user_defined_method(type(obj), '__call__')", "inspect.getdoc(_signature_get_user_defined_method)", "env=g", expand=10, env=g)

_signature_from_callable(Foo, sigcls=inspect.Signature, follow_wrapper_chains=True) # debug the function above through this example

        call = _signature_get_user_defined_method(type(obj), '__call__')=====================================================================================
                                                                                                                                              this is comment
                    return sigcls.from_callable(object)                                                                                                      
                else:                                                                                                                                        
                    raise ValueError(                                                                                                                        
                        'no signature found for builtin type {!r}'.format(obj))                                                                              
                                                    

<Signature (a, b, c)>

In [ ]:
sfc = dbprint(_signature_from_callable, "new = _signature_get_user_defined_method(obj, '__new__')", "this is comment", \
              "obj", "_signature_get_user_defined_method(obj, '__new__')", "inspect.getdoc(_signature_get_user_defined_method)", "env=g", expand=10, env=g)

_signature_from_callable(Foo, sigcls=inspect.Signature, follow_wrapper_chains=True) # debug the function above through this example

    if isinstance(obj, type):                                                                                                                                
        # obj is a class or a metaclass                                                                                                                      
                                                                                                                                                             
        # First, let's see if it has an overloaded __call__ defined                                                                                          
        # in its metaclass                                                                                                                                   
        call = _signature_get_user_defined_method(type(obj), '__call__')                                                                                     
        if call is not None:                        

<Signature (a, b, c)>

In [ ]:
defaults.orisrc??

Signature:
defaults.orisrc(
    obj,
    *,
    follow_wrapper_chains=True,
    skip_bound_arg=True,
    sigcls,
)
Source:   
def _signature_from_callable(obj, *,
                             follow_wrapper_chains=True,
                             skip_bound_arg=True,
                             sigcls):

    """Private helper function to get signature for arbitrary
    callable objects.
    """

    _get_signature_of = functools.partial(_signature_from_callable,
                                follow_wrapper_chains=follow_wrapper_chains,
                                skip_bound_arg=skip_bound_arg,
                                sigcls=sigcls)

    if not callable(obj):
        raise TypeError('{!r} is not a callable object'.format(obj))

    if isinstance(obj, types.MethodType):
        # In this case we skip the first parameter of the underlying
        # function (usually `self` or `cls`).
        sig = _get_signature_of(obj.__func__)

        if skip_bound_arg:
            ret